## Plotting Candle Sticks

Welcome to this ipython notebook for plotting candle sticks. We are going to use numpy, quandl and plotly in order to acquire data and plot out candle sticks

In [2]:
from datetime import datetime
import plotly
import plotly.offline as py
import plotly.graph_objs as go
import pandas as pd
import quandl
import talib

In [3]:
import talib
import numpy as np
import numpy as np
import torch
import torch.nn as nn
import matplotlib.pyplot as plot

Here we can specify the specific stock info that we want. Search the quandl free api calls here(https://www.quandl.com/search?query=) Currently I have a call bellow that gets bitcoin prices on a day resolution. We get the Open, Close, High, Low, Volume(BTC), Volume(Currency) and weighted price values for each day. These are stored in a pandas data frame and can be accesed i.e. stock_val['open']. All of the data has a date value associated with it.

In [4]:
stock_val=quandl.get("BCHARTS/BITFINEXUSD")

In [5]:
stock_val.describe()

,Open,High,Low,Close,Volume (BTC),Volume (Currency),Weighted Price
count,1363.000000,1363.000000,1363.000000,1363.000000,1363.000000,1.363000e+03,1363.000000
mean,405.304763,416.560585,392.450111,405.903719,18877.630270,7.472261e+06,404.871663
std,213.629493,219.881864,206.824893,213.810747,23524.240028,9.983940e+06,212.504780
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000
25%,236.600000,241.115000,233.200000,236.740000,5622.603863,2.144966e+06,237.142362
50%,389.980000,402.680000,376.400000,389.960000,11443.655525,4.390500e+06,388.345041
75%,586.755000,598.490000,573.970000,586.577550,23162.294126,9.226900e+06,586.824547
max,1142.990000,1175.000000,1090.000000,1143.000000,274465.728234,1.222903e+08,1137.097669


Next we are going to make a nice candle stick plot of the data. In order to do this you have to setup an account at plotly(https://plot.ly). After you setup an account make sure to confiure your account according to these directions(https://plot.ly/python/getting-started/). Once this is done you should be able to run the bellow and get out a simple candle stick chart. Check out other charts here(https://plot.ly/python/candlestick-charts/)

In [6]:
trace = go.Candlestick(x=stock_val.index,
                       open=stock_val.Open,
                       high=stock_val.High,
                       low=stock_val.Low,
                       close=stock_val.Close)
data = [trace]
py.iplot(data, filename='simple_candlestick')

High five! You successfully sent some data to your account on plotly. View your plot in your browser at https://plot.ly/~lthul/0 or inside your plot.ly account where it is named 'simple_candlestick'


In [7]:
candles={}
candle_cnts={'x':[], 'y':[]}
time_vals=stock_val.index
candle_stick_patterns=talib.get_function_groups()['Pattern Recognition']
for pat in candle_stick_patterns:
    candles[pat]=getattr(talib,pat)(stock_val['Open'], stock_val['High'], stock_val['Low'], stock_val['Close'])
    candle_cnts['x'].append(pat)
    candle_cnts['y'].append(len(np.where(candles[pat]>0)[0]))
candles_bar = pd.DataFrame(data=candle_cnts)

Create bar chart of the counts for each type of candlestick

In [33]:
candle_bar_plt = [go.Bar(
            x=candles_bar['x'],
            y=candles_bar['y']
    )]
layout = go.Layout(
    title='Counts of each candlestick type',
    xaxis=dict(
        title='Candlestick types',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
    ),
    yaxis=dict(
        title='Counts',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
    )
)
py.iplot({'data': candle_bar_plt, 'layout': layout}, filename='basic-bar')

Defining a heatmap of candlestick activity

In [34]:
trace = go.Heatmap(x=time_vals.tolist(),
                y=list(candles.keys())[0:20],   # there are 61 candlesticks, any more than 10 at a time
                z=list(candles.values())[0:20]) # isn't very useful
data=[trace]
layout = go.Layout(title='Candlestick Heatmap',
    margin=go.Margin(
        l=160, # for left margin cutoff
        r=50,
        b=50,
        t=100,
        pad=4
    ))
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='candlestick-heatmap')

In [8]:
stock_val.values

array([[9.32500000e+01, 1.00000000e+02, 9.30300000e+01, ...,
        3.90827224e+02, 3.70893330e+04, 9.48995634e+01],
       [9.31700000e+01, 1.05900000e+02, 9.24999900e+01, ...,
        4.91965413e+03, 4.92101017e+05, 1.00027564e+02],
       [1.02800000e+02, 1.18388067e+02, 9.90000000e+01, ...,
        9.08483282e+03, 9.87411878e+05, 1.08687953e+02],
       ...,
       [7.90590000e+02, 8.02330000e+02, 7.87860000e+02, ...,
        6.99255929e+03, 5.55029679e+06, 7.93743258e+02],
       [7.97990000e+02, 8.30500000e+02, 7.94950000e+02, ...,
        1.62166405e+04, 1.32357307e+07, 8.16182038e+02],
       [8.29340000e+02, 8.74000000e+02, 8.25000000e+02, ...,
        1.08575574e+04, 9.27245156e+06, 8.54008981e+02]])